In [1]:
%run ./packer.ipynb

In [2]:
import pandas as pd
import numpy as np
import csv
import time
import random
import matplotlib.pyplot as plt
from gurobipy import *  
import math
import scipy.interpolate as interp
from mpl_toolkits import mplot3d
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from matplotlib import animation

In [4]:
data = pd.read_excel('Data.xlsx')
tray = pd.read_excel('TrayData.xlsx')

In [5]:
data

,MODEL,cl,cw,ch,Cost,Cap
0,S-19040,8.0,6.0,4.0,0.31,30
1,S-22174,9.0,6.0,3.0,0.33,30
2,S-16701,8.0,5.0,3.0,0.34,30
3,S-4593,9.0,5.0,3.0,0.34,30
4,S-4556,8.0,5.0,4.0,0.35,30
...,...,...,...,...,...,...
1341,S-16462▪,32.0,32.0,32.0,14.96,30
1342,S-20991▪,24.0,24.0,24.0,16.72,30
1343,S-18044▪,24.0,22.0,60.0,20.33,30
1344,S-13331▪,24.0,24.0,24.0,22.00,30


In [31]:
ABCDE = tuplelist(zip(tray["Tray"],tray["L"],tray["W"],tray["H"],tray["weight"]))

In [32]:
len(ABCDE)

4

In [16]:
IJKLM = tuplelist(zip(data["MODEL"],data["cl"],data["cw"],data["ch"],data["Cap"]))

In [19]:
len(IJKLM)

1346

In [33]:
ABCDE

<gurobi.tuplelist (4 tuples, 5 values each):
 ( a , 3.156 , 3.156 , 1.531  , 0.19 )
 ( b , 4.375 , 4.375 , 1.7375 , 0.38 )
 ( c , 4.375 , 4.375 , 2.2188 , 1.35 )
 ( d , 4.625 , 4.625 , 2.594  , 0.75 )
>

In [40]:
b = list(range(0,3))

In [28]:
a = list(range(0,1345))

In [50]:
IJKLM = list(IJKLM)
ABCDE = list(ABCDE)

In [51]:
for i in a:
    print(IJKLM[i])

('S-19040', 8.0, 6.0, 4.0, 30)
('S-22174', 9.0, 6.0, 3.0, 30)
('S-16701', 8.0, 5.0, 3.0, 30)
('S-4593', 9.0, 5.0, 3.0, 30)
('S-4556', 8.0, 5.0, 4.0, 30)
('S-4705', 8.0, 6.0, 3.0, 30)
('S-16705', 9.0, 5.0, 4.0, 30)
('S-19864', 9.0, 6.0, 4.0, 30)
('S-4308', 8.0, 5.0, 5.0, 30)
('S-4080', 8.0, 6.0, 4.0, 30)
('S-22175', 9.0, 7.0, 4.0, 30)
('S-19041', 8.0, 6.0, 6.0, 30)
('S-4092', 9.0, 5.0, 5.0, 30)
('S-19865', 9.0, 6.0, 6.0, 30)
('S-19867', 9.0, 7.0, 5.0, 30)
('S-23302', 10.0, 5.0, 5.0, 30)
('S-19060', 10.0, 6.0, 4.0, 30)
('S-4949', 9.0, 6.0, 3.0, 30)
('S-19061', 10.0, 6.0, 6.0, 30)
('S-19861', 12.0, 6.0, 4.0, 30)
('S-4484', 9.0, 6.0, 4.0, 30)
('S-21587', 9.0, 9.0, 4.0, 30)
('S-19043', 8.0, 8.0, 4.0, 30)
('S-22176', 9.0, 9.0, 5.0, 30)
('S-22629', 10.0, 7.0, 5.0, 30)
('S-19063', 12.0, 6.0, 6.0, 30)
('S-18336', 8.0, 8.0, 8.0, 30)
('S-19100', 9.0, 7.0, 3.0, 30)
('S-4600', 10.0, 6.0, 3.0, 30)
('S-21012', 10.0, 8.0, 4.0, 30)
('S-4515', 8.0, 6.0, 5.0, 30)
('S-4707', 9.0, 6.0, 5.0, 30)
('S-4594', 

In [62]:
str(IJKLM[0])

"('S-19040', 8.0, 6.0, 4.0, 30)"

In [64]:
packer = Packer()
for i in a:
    packer.add_bin(Bin(*IJKLM[i]))
# packer.add_bin(Bin('large-envelope', 15.0, 12.0, 0.75, 15))
# packer.add_bin(Bin('small-box', 8.625, 5.375, 1.625, 70.0))
# packer.add_bin(Bin('medium-box', 11.0, 8.5, 5.5, 70.0))
# packer.add_bin(Bin('medium-2-box', 13.625, 11.875, 3.375, 70.0))
# packer.add_bin(Bin('large-box', 12.0, 12.0, 5.5, 70.0))
# packer.add_bin(Bin('large-2-box', 23.6875, 11.75, 3.0, 70.0))
for j in b:
    packer.add_item(Item(*ABCDE[j]))
# packer.add_item(Item('50g [powder 2]', 3.9370, 1.9685, 1.9685, 2))
# packer.add_item(Item('50g [powder 3]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 4]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 5]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 6]', 3.9370, 1.9685, 1.9685, 3))
# packer.add_item(Item('50g [powder 7]', 5.1240, 1.1350, 1.5435, 3))
# packer.add_item(Item('250g [powder 8]', 7.8740, 3.9370, 1.9685, 4))
# packer.add_item(Item('250g [powder 9]', 7.8740, 3.9370, 1.9685, 5))
# packer.add_item(Item('250g [powder 10]', 7.8740, 3.9370, 1.9685, 6))
# packer.add_item(Item('250g [powder 11]', 7.8740, 3.9370, 1.9685, 7))
# packer.add_item(Item('250g [powder 12]', 7.8740, 3.9370, 1.9685, 8))
# packer.add_item(Item('250g [powder 13]', 7.8740, 3.9370, 1.9685, 9))

packer.pack()


::::::::::: S-16701(8.000x5.000x3.000, max_weight:30.000) vol(120.000) item_number(2) filling_ratio(0.481)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.375'), 0, 0]) rt(0) vol(15.249)

UNFITTED ITEMS:
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(5) vol(33.266)

::::::::::: S-4593(9.000x5.000x3.000, max_weight:30.000) vol(135.000) item_number(2) filling_ratio(0.561)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('4.375'), 0, 0]) rt(0) vol(33.266)

UNFITTED ITEMS:
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.375'), 0, Decimal('1.738')]) rt(5) vol(15.249)

::::::::::: S-4705(8.000x6.000x3.000, max_weight:30.000) vol(144.000) item_number(2) filling_ratio(0.401)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.

FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(0) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.375'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-23290(12.000x12.000x3.000, max_weight:30.000) vol(432.000) item_number(3) filling_ratio(0.211)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('4.375'), 0, 0]) rt(0) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(0) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4521(12.000x9.000x4.000, max_weight:30.000) vol(432.000) item_number(3) filling_ratio(0.211)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(0) vol(33.266)
====>  a(3.156x3.156x1.

::::::::::: S-4596(9.000x9.000x7.000, max_weight:30.000) vol(567.000) item_number(3) filling_ratio(0.160)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(3) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-19064(12.000x8.000x6.000, max_weight:30.000) vol(576.000) item_number(3) filling_ratio(0.158)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-16706(9.000x8.000x8.000, max_weight:30.000) vol(576.000) item_number(3) filling_ratio(0.158)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(

====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(3) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-23301(12.000x12.000x5.000, max_weight:30.000) vol(720.000) item_number(3) filling_ratio(0.126)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(3) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4639(20.000x6.000x6.000, max_weight:30.000) vol(720.000) item_number(3) filling_ratio(0.126)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::

::::::::::: S-11371(11.000x9.000x9.000, max_weight:30.000) vol(891.000) item_number(3) filling_ratio(0.102)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(0) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4597(9.000x9.000x11.000, max_weight:30.000) vol(891.000) item_number(3) filling_ratio(0.102)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.375'), 0, 0]) rt(0) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-21006(14.000x8.000x8.000, max_weight:30.000) vol(896.000) item_number(3) filling_ratio(0.102)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  

...

::::::::::: S-4808(29.000x17.000x12.000, max_weight:30.000) vol(5916.000) item_number(3) filling_ratio(0.015)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4667(30.000x20.000x10.000, max_weight:30.000) vol(6000.000) item_number(3) filling_ratio(0.015)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4862(20.000x20.000x15.000, max_weight:30.000) vol(6000.000) item_number(3) filling_ratio(0.015)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)


::::::::::: S-18321▪(30.000x6.000x40.000, max_weight:30.000) vol(7200.000) item_number(3) filling_ratio(0.013)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(0) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.957')]) rt(0) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-18371(26.000x20.000x14.000, max_weight:30.000) vol(7280.000) item_number(3) filling_ratio(0.012)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4701(29.000x17.000x15.000, max_weight:30.000) vol(7395.000) item_number(3) filling_ratio(0.012)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473

====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-18168▪(28.000x18.000x18.000, max_weight:30.000) vol(9072.000) item_number(3) filling_ratio(0.010)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('3.957'), 0, 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4676(18.000x18.000x28.000, max_weight:30.000) vol(9072.000) item_number(3) filling_ratio(0.010)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(0) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, 0, Decimal('3.957')]) rt(0) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-23331(24.000x24.000x16.000, max_weight:30.000) vol(9216.000) item_number(3)

FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(3) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4911▪(25.000x25.000x20.000, max_weight:30.000) vol(12500.000) item_number(3) filling_ratio(0.007)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(3) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([0, Decimal('4.375'), 0]) rt(1) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-19824▪(24.000x24.000x22.000, max_weight:30.000) vol(12672.000) item_number(3) filling_ratio(0.007)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(1) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([Decimal('2.219'), 0, 0]) rt(3) vol(33.266)
====>  a(3.156

::::::::::: S-4433▪(28.000x28.000x28.000, max_weight:30.000) vol(21952.000) item_number(3) filling_ratio(0.004)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.375'), 0, 0]) rt(0) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-16461▪(28.000x28.000x28.000, max_weight:30.000) vol(21952.000) item_number(3) filling_ratio(0.004)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(42.473)
====>  b(4.375x4.375x1.738, weight: 0.380) pos([0, 0, Decimal('2.219')]) rt(1) vol(33.266)
====>  a(3.156x3.156x1.531, weight: 0.190) pos([Decimal('4.375'), 0, 0]) rt(0) vol(15.249)

UNFITTED ITEMS:

::::::::::: S-4750▪(24.000x20.000x46.000, max_weight:30.000) vol(22080.000) item_number(3) filling_ratio(0.004)
FITTED ITEMS:
====>  c(4.375x4.375x2.219, weight: 1.350) pos([0, 0, 0]) rt(0) vol(4